In [3]:
import math 
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [26]:
class Value:
    def __init__(self, data, _children=(), _op='', label='') -> None:
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = _children
        self._op = _op
        self.label = label
    
    def __repr__(self) -> str:
        return f'Value(data = {self.data})'
    
    

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self,other), '+')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward

        return out
    
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self,other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out
    

    def __pow__(self, other):
        assert isinstance(other, (int,float)) # only support int & float powers for now
        out = Value(self.data**other, (self,), f'^{other}')

        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward

        return out
    
    
    def __neg__(self):
        return -1 * self
    
    def __sub__(self, other):
        return self + (-other)
    
    def __rsub__(self, other):
        return self + (-other)
    
    def __radd__(self, other):
        return self + other
    
    def __rmul__(self, other):
        return self * other
    
    def __truediv__(self, other):
        return self * (other ** -1)
    
    def __rtruediv__(self, other):
        return self * (other ** -1)

    
    def sigmoid(self):
        x = self.data
        s = 1/(1+ np.exp(-x))
        out = Value(s, (self, ), 'sigmoid')

        def _backward():
            self.grad += s * (1-s)

        out._backward = _backward
        
        return out
    
    def relu(self):
        x = self.data
        r = max(x,0)
        out = Value(r, (self, ), 'ReLU')

        def _backward():
            self.grad += (out.grad if x > 0 else 0)

        out._backward = _backward

        return out
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')
        
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        
        return out
    
    def linear(self):
        x = self.data
        out = Value(x, (self, ), 'linear')

        def _backward():
            self.grad += 1
        out._backward = _backward
        
        return out
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')

        def _backward():
            self.grad += out.grad * out.data
        out._backward = _backward
        return out
    

    def backward(self):
        topological = []
        visited = set()

        def build_topological(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topological(child)
                topological.append(v)

        build_topological(self)
        
        self.grad = 1.0
        for node in reversed(topological):
            node._backward()

In [51]:
class Neuron:
    
    def __init__(self, numIn) -> None:
        self.w = [Value(random.uniform(-1,1)) for _ in range(numIn)]
        self.b = Value(random.uniform(-1,1))
    
    def __call__(self, x):
        # w . x + b
        act = sum(w1*x1 for w1, x1 in zip(self.w, x)) + self.b
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]

class Layer:
    def __init__(self, numIn, numOut) -> None:
        self.neurons = [Neuron(numIn) for _ in range(numOut)]
    
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        params = []
        for neuron in self.neurons:
            params.extend(neuron.parameters())
        return params
    
        
class MLP:
    def __init__(self, numIn, numOuts) -> None:
        sizes = [numIn] + numOuts 
        self.layers = [Layer(sizes[i], sizes[i+1]) for i in range(len(numOuts))]
    
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        params = []
        for layer in self.layers:
            params.extend(layer.parameters())
        return params
        

In [55]:
import pandas as pd
df = pd.read_csv('data.csv', header = 0)

df.drop('area_se', axis=1, inplace=True)
df.drop('area_mean', axis=1, inplace=True)
df.drop('perimeter_worst', axis=1, inplace=True)
df.drop('area_worst', axis=1, inplace=True)
df.drop('perimeter_mean', axis=1, inplace=True)


df = df.iloc[:, 1:]


y = df.iloc[:, 0].tolist()  # First column
X_train = df.iloc[:, 1:].values.tolist()  # Remaining columns

for x in X_train:
    x.pop()

y_train = [1 if x == 'M' else -1 for x in y]

# xs -> 29-ary arrays 
# ys -> labels  

In [56]:
n = MLP(3, [15, 10, 1])

In [ ]:
# Forward pass
ypred = [n(x) for x in X_train]
loss = sum((yout - ygt)**2 for ygt, yout in zip(y_train, ypred))

# Backward pass
for p in n.parameters():
    p.grad = 0
loss.backward()

# Update
for p in n.parameters():
    p.data -= 0.1 * p.grad

print (loss.data)



256.0


In [53]:
# Using MSE loss

for k in range(20):

    # Forward pass
    ypred = [n(x) for x in X_train]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(y_train, ypred))

    # Backward pass
    for p in n.parameters():
        p.grad = 0
    loss.backward()

    # Update
    for p in n.parameters():
        p.data -= 0.1 * p.grad

    print (k, loss.data)

ypred

0 99.00000037532686
1 99.00000037531683
2 99.00000037530677
3 99.00000037529668
4 99.00000037528663
5 99.00000037527658
6 99.00000037526652
7 99.00000037525649
8 99.00000037524644
9 99.0000003752364
10 99.00000037522635
11 99.00000037521627
12 99.00000037520623
13 99.00000037519615
14 99.00000037518612
15 99.00000037517606
16 99.00000037516604
17 99.00000037515599
18 99.00000037514592
19 99.00000037513585


[Value(data = 7.463791488355264e-10),
 Value(data = 3.8601148093364575e-14),
 Value(data = 3.717163529153519e-15),
 Value(data = 3.044462358058028e-11),
 Value(data = 2.1098926918399133e-12),
 Value(data = 1.1669960710065768e-10),
 Value(data = 3.2201420726449936e-14),
 Value(data = 1.1980982460046766e-12),
 Value(data = 1.5516157270094184e-12),
 Value(data = 1.4671195152691513e-12),
 Value(data = 2.0171214871369303e-14),
 Value(data = 1.1835757867412184e-12),
 Value(data = 2.1478525468126465e-16),
 Value(data = 1.579641686380696e-14),
 Value(data = 3.933418666315616e-13),
 Value(data = 1.5292558082270733e-14),
 Value(data = 6.191005155893421e-13),
 Value(data = 8.433483404910953e-14),
 Value(data = 1.4525455820748731e-15),
 Value(data = 1.5056044260068125e-10),
 Value(data = 5.7335544119705674e-11),
 Value(data = 2.4849899042575405e-08),
 Value(data = 5.095981587318489e-11),
 Value(data = 2.587230937067063e-16),
 Value(data = 3.0472649498895596e-14),
 Value(data = 2.0533694172729067e-